# RSA

In [1]:
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes
from icecream import ic
from sage.all import *

## Overview
### Parameters
- `p` & `q`: prime numbers that are kept secret
- `n`: public modulus (`n = p*q`)
- `e`: public exponent, typically 3 or 65537
- 'l': `l = (p-1) * (q-1)`
- `d`: private exponent  `d = pow(e, -1, l)`
- d_p & d_q, private exponent parts (used to speed up calculations sometimes)
  - d_p = d_p*e = 1 (mod p-1)
  - d_q = d_q*e = 1 (mod q-1)

In [2]:
# Basic encryption and decryption
e = 3
while True:
    p = getPrime(64)
    q = getPrime(64)
    n = p*q

    l = (p-1) * (q-1)
    if gcd(e, l) != 1:
        continue # e and l must be coprime for a unique decryption
    d = pow(e, -1, l)
    break

pt = int.from_bytes(b'Sam is cool', 'big')

# Encryption
ct = pow(pt, e, n)

# Decryption
pt2 = int(pow(ct, d, n))

ic(pt)
ic(ct)
ic(pt2)
long_to_bytes(pt2)

ic| pt: 100800925607481076140175212
ic| ct: 143060088881062105511120140810776844571
ic| pt2: 100800925607481076140175212


b'Sam is cool'

In [4]:
"""
Like Håstad's and Franklin-Reiter's attacks, this attack exploits a weakness of RSA with public exponent e=3.
Coppersmith showed that if randomized padding suggested by Håstad is used improperly, then RSA encryption is not secure.

Conditions:
e=3 and m<=n/e^2
"""

from typing import List
from sage.all import *




def attack(ct1: int, ct2: int):
    # https://en.wikipedia.org/wiki/Coppersmith%27s_attack#Coppersmith%E2%80%99s_short-pad_attack
    # https://github.com/ValarDragon/CTF-Crypto/blob/master/RSA/FranklinReiter.sage

    # Inputs are modulus, known difference, ciphertext 1, ciphertext2.
    # Ciphertext 1 corresponds to smaller of the two plaintexts. (The one without the fixed difference added to it)
    def franklinReiter(n,e,r,c1,c2):
        # R.<X> = Zmod(n)[]
        R = Zmod(n)['X']; (X,) = R._first_ngens(1)
        f1 = X^e - c1
        f2 = (X + r)^e - c2
        # coefficient 0 = -m, which is what we wanted!
        return Integer(n-(compositeModulusGCD(f1,f2)).coefficients()[0])

    # GCD is not implemented for rings over composite modulus in Sage
    # so we do our own implementation. Its the exact same as standard GCD, but with
    # the polynomials monic representation
    def compositeModulusGCD(a, b):
        if(b == 0):
            return a.monic()
        else:
            return compositeModulusGCD(b, a % b)


    e = 3

    # P.<x,y> = PolynomialRing(ZZ)
    P = PolynomialRing(ZZ, names=('x', 'y',)); (x, y,) = P._first_ngens(2)
    ZmodN = Zmod(n)
    g1 = x**e - ct1
    g2 = (x+y)**e - ct2
    res = g1.resultant(g2)
    # P.<y> = PolynomialRing(ZmodN)
    P = PolynomialRing(ZmodN, names=('y',)); (y,) = P._first_ngens(1)

    # Convert Multivariate Polynomial Ring to Univariate Polynomial Ring
    rres = 0
    for i in range(len(res.coefficients())):
        rres += res.coefficients()[i]*(y**(res.exponents()[i][1]))

    epsilon=1/30
    diff = rres.small_roots(epsilon=epsilon)
    recovered_m1 = franklinReiter(n, e, diff[0], ct1, ct2)

    # Message could be right shifted by up to 7 bits
    possible_messages = []  # type: List[int]
    for i in range(8):
        possible_messages.append(recovered_m1 << 1)